In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('tablaminable.csv')
data = data.drop('c.client_id', axis=1)
X = data.drop('l.status', axis=1)
y = data['l.status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo KNN: {accuracy}")

f1 = f1_score(y_test, y_pred, average='micro')
print(f"F1-score del modelo KNN: {f1}")

# Calcular el AUC para clases múltiples (One-vs-Rest)
auc_scores_ovr = roc_auc_score(y_test, knn.predict_proba(X_test), multi_class='ovr')

print(f"AUC One-vs-Rest: {auc_scores_ovr}")


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

data = pd.read_csv('tablaminable.csv')
data = data.drop('c.client_id', axis=1)
X = data.drop('l.status', axis=1)
y = data['l.status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(X_train, y_train)

y_pred = tree_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo Decision Tree: {accuracy}")

f1 = f1_score(y_test, y_pred, average='micro')
print(f"F1-score del modelo Decision Tree: {f1}")

# Calcular el AUC para clases múltiples (One-vs-Rest)
auc_scores_ovr = roc_auc_score(y_test, tree_classifier.predict_proba(X_test), multi_class='ovr')

print(f"AUC One-vs-Rest: {auc_scores_ovr}")



In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

data = pd.read_csv('tablaminable.csv')
data = data.drop('c.client_id', axis=1)
X = data.drop('l.status', axis=1)
y = data['l.status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)

y_pred = logistic_regression.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo de Regresión Logística: {accuracy}")

f1 = f1_score(y_test, y_pred, average='micro')
print(f"F1-score del modelo Regresión Logística: {f1}")

# Calcular el AUC para clases múltiples (One-vs-Rest)
auc_scores_ovr = roc_auc_score(y_test, logistic_regression.predict_proba(X_test), multi_class='ovr')

print(f"AUC One-vs-Rest: {auc_scores_ovr}")
